# Getting Start

In [1]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd
import time

In [2]:
api_key = "TGV2OZgIKqiCEWvhVUAUDGBB99Eixo1z2ILlZ0h3lRdutfklTwRlqzPYpIazsYcn"
secret_key = "CrgLtKo6MRxdL6tJRNKokKPKOAWa0Rn59Q9KfX57yal0E1WTqz79jk2IT03g6VcZ"
#test api_key
test_api_Key = "ni1RGarrqKhXFMnmqZt2EEbEJkpGDXs4AxHml3r7aevzg7uqUR1rTFbNngWiTOln"
test_secret_Key = "HqFm7zQNQPOEJdYMgLho7engJbah6FAH98Eqf607h467CB9RM9e6dHDytnQbKGgk"

In [3]:
client = Client(api_key = test_api_Key, api_secret = test_secret_Key, tld = "com", testnet = True)

In [4]:
client

In [5]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1710388795202,
 'accountType': 'SPOT',
 'balances': [{'asset': 'ETH', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '5.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '1.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '3798.00000000', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '727.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '27.00000000', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '91.0000000

### Option 1: ThreadedWebsocketManager (only executable in scripts, not in Jupyter!):

In [6]:
twm = ThreadedWebsocketManager()
twm.start()

In [7]:
df = pd.DataFrame(columns = ["Open", "High", "Low", "Close", "Volume", "Complete"])
df

,Open,High,Low,Close,Volume,Complete


In [8]:
def stream_candles(msg):
    ''' define how to process incoming WebSocket messages '''
    
    # extract the required items from msg
    event_time = pd.to_datetime(msg["E"], unit = "ms")
    start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
    first   = float(msg["k"]["o"])
    high    = float(msg["k"]["h"])
    low     = float(msg["k"]["l"])
    close   = float(msg["k"]["c"])
    volume  = float(msg["k"]["v"])
    complete=       msg["k"]["x"]
    
    # print out
    print("Time: {} | Price: {}".format(event_time, close))
    # feed df (add new bar / update latest bar)
    df.loc[start_time] = [first, high, low, close, volume, complete]

In [9]:
twm.start_kline_socket(callback = stream_candles, symbol = "BTCUSDT", interval = "1m")

'btcusdt@kline_1m'

Time: 2024-03-26 08:23:24.679000 | Price: 70636.0


In [10]:
while True:
    time.sleep(120) # stop after 2 minutes
    twm.stop()
    break

Time: 2024-03-26 08:23:26.829000 | Price: 70640.99
Time: 2024-03-26 08:23:28.930000 | Price: 70646.0
Time: 2024-03-26 08:23:30.953000 | Price: 70653.24
Time: 2024-03-26 08:23:33.080000 | Price: 70645.49
Time: 2024-03-26 08:23:35.165000 | Price: 70645.49
Time: 2024-03-26 08:23:37.679000 | Price: 70645.5
Time: 2024-03-26 08:23:39.702000 | Price: 70652.49
Time: 2024-03-26 08:23:41.709000 | Price: 70681.09
Time: 2024-03-26 08:23:43.710000 | Price: 70667.19
Time: 2024-03-26 08:23:45.861000 | Price: 70656.41
Time: 2024-03-26 08:23:48.377000 | Price: 70664.4
Time: 2024-03-26 08:23:50.535000 | Price: 70673.99
Time: 2024-03-26 08:23:52.590000 | Price: 70664.26
Time: 2024-03-26 08:23:54.761000 | Price: 70664.26
Time: 2024-03-26 08:23:56.845000 | Price: 70656.42
Time: 2024-03-26 08:23:59.006000 | Price: 70650.0
Time: 2024-03-26 08:24:00.007000 | Price: 70650.01
Time: 2024-03-26 08:24:02.443000 | Price: 70644.18
Time: 2024-03-26 08:24:04.676000 | Price: 70655.92
Time: 2024-03-26 08:24:06.696000 | 

In [11]:
df

,Open,High,Low,Close,Volume,Complete
2024-03-26 08:23:00,70684.99,70685.00,70627.58,70650.01,30.31457,True
2024-03-26 08:24:00,70650.00,70684.00,70632.00,70632.00,11.16508,True
2024-03-26 08:25:00,70632.01,70643.68,70611.83,70643.68,7.98001,False


### Option 2: BinanceSocketManager (executable in Jupyter!): 

In [12]:
import asyncio
from binance import AsyncClient, BinanceSocketManager

In [13]:
df = pd.DataFrame(columns = ["Open", "High", "Low", "Close", "Volume", "Complete"])
df

,Open,High,Low,Close,Volume,Complete


In [14]:
stream_candles

<function __main__.stream_candles(msg)>

In [16]:
async def main():
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client)
    ts = bm.kline_socket(symbol = "BTCUSDT", interval = "1m")
    
    async with ts as tscm:
        for _ in range(120):  # This is just an example to limit the number of messages. Remove or adjust as needed.
            res = await tscm.recv()
            stream_candles(res)
            print("res"+str(res))

    await client.close_connection()
await main()

Time: 2024-03-26 08:28:38.849000 | Price: 70594.0
res{'e': 'kline', 'E': 1711441718849, 's': 'BTCUSDT', 'k': {'t': 1711441680000, 'T': 1711441739999, 's': 'BTCUSDT', 'i': '1m', 'f': 3514431371, 'L': 3514432465, 'o': '70613.30000000', 'c': '70594.00000000', 'h': '70615.80000000', 'l': '70567.17000000', 'v': '17.91802000', 'n': 1095, 'x': False, 'q': '1264815.16431050', 'V': '8.65108000', 'Q': '610656.94956250', 'B': '0'}}
Time: 2024-03-26 08:28:40.945000 | Price: 70607.9
res{'e': 'kline', 'E': 1711441720945, 's': 'BTCUSDT', 'k': {'t': 1711441680000, 'T': 1711441739999, 's': 'BTCUSDT', 'i': '1m', 'f': 3514431371, 'L': 3514432538, 'o': '70613.30000000', 'c': '70607.90000000', 'h': '70615.80000000', 'l': '70567.17000000', 'v': '18.36537000', 'n': 1168, 'x': False, 'q': '1296398.02785210', 'V': '9.08703000', 'Q': '641434.92081570', 'B': '0'}}
Time: 2024-03-26 08:28:42.988000 | Price: 70607.9
res{'e': 'kline', 'E': 1711441722988, 's': 'BTCUSDT', 'k': {'t': 1711441680000, 'T': 1711441739999, 

In [17]:
df

,Open,High,Low,Close,Volume,Complete
2024-03-26 08:28:00,70613.30,70615.80,70567.17,70608.55,20.97720,True
2024-03-26 08:29:00,70608.56,70613.32,70569.69,70577.64,17.00060,True
2024-03-26 08:30:00,70577.65,70620.00,70571.65,70589.28,27.16815,True
2024-03-26 08:31:00,70589.28,70600.00,70477.35,70539.99,47.00776,True
2024-03-26 08:32:00,70540.00,70544.90,70483.99,70512.00,17.76602,False


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5 entries, 2024-03-26 08:28:00 to 2024-03-26 08:32:00
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Open      5 non-null      float64
 1   High      5 non-null      float64
 2   Low       5 non-null      float64
 3   Close     5 non-null      float64
 4   Volume    5 non-null      float64
 5   Complete  5 non-null      bool   
dtypes: bool(1), float64(5)
memory usage: 417.0 bytes


# **A LongOnlyTrader Class**

In [19]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length):
        self.symbol = symbol
        self.bar_length = bar_length
        self.data = pd.DataFrame(columns = ["Open", "High", "Low", "Close", "Volume", "Complete"])
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
    
    def start_trading(self):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print("Time: {} | Price: {}".format(event_time, close))
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]

In [20]:
trader = LongOnlyTrader(symbol = "BTCUSDT", bar_length = "1m")
trader

In [21]:
trader.symbol

'BTCUSDT'

In [22]:
trader.data

,Open,High,Low,Close,Volume,Complete


In [23]:
trader.available_intervals

['1m',
 '3m',
 '5m',
 '15m',
 '30m',
 '1h',
 '2h',
 '4h',
 '6h',
 '8h',
 '12h',
 '1d',
 '3d',
 '1w',
 '1M']

In [24]:
trader.start_trading()

Time: 2024-03-26 10:02:10.710000 | Price: 71179.22
Time: 2024-03-26 10:02:12.814000 | Price: 71179.22
Time: 2024-03-26 10:02:14.957000 | Price: 71179.21
Time: 2024-03-26 10:02:16.991000 | Price: 71174.01
Time: 2024-03-26 10:02:19.052000 | Price: 71177.99
Time: 2024-03-26 10:02:21.336000 | Price: 71189.08
Time: 2024-03-26 10:02:23.502000 | Price: 71189.09
Time: 2024-03-26 10:02:25.562000 | Price: 71186.52
Time: 2024-03-26 10:02:27.594000 | Price: 71182.72
Time: 2024-03-26 10:02:29.611000 | Price: 71182.72
Time: 2024-03-26 10:02:31.622000 | Price: 71182.71
Time: 2024-03-26 10:02:33.632000 | Price: 71170.73
Time: 2024-03-26 10:02:35.886000 | Price: 71165.99
Time: 2024-03-26 10:02:38.560000 | Price: 71166.0
Time: 2024-03-26 10:02:41.054000 | Price: 71157.21
Time: 2024-03-26 10:02:43.298000 | Price: 71157.22
Time: 2024-03-26 10:02:45.331000 | Price: 71150.01
Time: 2024-03-26 10:02:47.377000 | Price: 71150.0
Time: 2024-03-26 10:02:49.524000 | Price: 71141.11
Time: 2024-03-26 10:02:51.736000 

In [25]:
trader.twm.stop()

Time: 2024-03-26 10:24:56.508000 | Price: 71001.0


In [26]:
trader.data

,Open,High,Low,Close,Volume,Complete
2024-03-26 10:02:00,71137.32,71189.09,71123.71,71123.72,23.00424,True
2024-03-26 10:03:00,71123.73,71134.43,71089.56,71102.25,18.95773,True
2024-03-26 10:04:00,71102.24,71117.20,71067.70,71082.00,34.97893,True
2024-03-26 10:05:00,71082.01,71086.00,71030.21,71068.73,23.59248,True
2024-03-26 10:06:00,71068.72,71085.99,71053.20,71066.77,29.22708,True
2024-03-26 10:07:00,71066.77,71066.77,71034.24,71043.74,17.46717,True
2024-03-26 10:08:00,71043.74,71079.24,71032.00,71045.99,33.23859,True
2024-03-26 10:09:00,71046.00,71075.60,71038.00,71066.78,13.41250,True
2024-03-26 10:10:00,71066.78,71121.10,71066.77,71112.23,14.11865,True
2024-03-26 10:11:00,71112.23,71139.73,71112.22,71139.73,6.91490,True
